In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from IPython.display import display
import pandas as pd
import h5py
import os
from tqdm import tqdm
# to load data from the test file
from utils.create_datasets import SumDatasets
# To load the data sets here, also we will create some examples to explore torch
data_dir = '../data/'
from utils import config
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

/home/chenxiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sum_dataset = SumDatasets(os.path.join(data_dir, 'features-600-40_v2.hdf5'))

In [5]:
def encoder_transform(features_1, features_2, features_3, features_4):
    """
    To transform the data to the format, we need to order the data in the sequences of length
    ,so we can later utilize the data in the NN model with the pack_paded_sequences and pad_packed_sequences.
    
    Parameters
    ----------
        
    """
    assert isinstance(features_1, torch.Tensor), "You must give the data to tensor object"
    batch_size = features_1.size(0)
    if batch_size == 1:
        pass
    else:
        sorted_length, sorted_idx = features_2.sort()  # sort will return both the ascending sorted value and also the sorted index
        reverse_idx = torch.linspace(batch_size - 1, 0, batch_size).long()  # this will contain the batch_size-1
        sorted_length, sorted_idx = sorted_length[reverse_idx], sorted_idx[reverse_idx]
        features_1 = features_1[sorted_idx]
        features_2 = features_2[sorted_idx]
        features_3 = features_3[sorted_idx]
        features_4 = features_4[sorted_idx]
    features_1.squeeze_(1), features_3.squeeze_(1), features_4.squeeze_(1)
    features_3 = features_3.permute([1, 0])
    features_4 = features_4.permute([1, 0])
    return features_1, features_2, features_3, features_4
features_1, features_2, features_3, features_4 = sum_dataset[0:10] # assume we have a batch data of 20 items.
display(features_2)

tensor([600, 600, 600, 600, 600, 600, 600, 600, 600, 436])

## given the data, build a very naive summarization model here
* first, we need to compute all the hidden state of each time step and a final state, contain cell state and hidden state of the final time and feed it into the network and do the decoder agian.

In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(config.NUM_WORDS + 2, config.embedding_dim)
        self.rnn = nn.LSTM(config.embedding_dim, config.hidden_dim, bidirectional=True)
        self.reduce_ = nn.Linear(2 * config.hidden_dim, config.hidden_dim)
        
    def forward(self, X, seq_lens):
        """return the final satets and also the outputs of each timesteps, for the later usage of
        computing the Attentaion matrix foe each time step input of the Decoder.
        
        Parameters
        ----------
        X : [Torch tensor with batch*MAX_STEP]
            
        seq_lens : [descend order of the real length of the data]
        """
        X = self.embed(X)
        batch_size = X.size()[0]
        packed_x = pack_padded_sequence(X, seq_lens, batch_first=True)
        outputs, hidden = self.rnn(packed_x)
        outputs, seq_lens = pad_packed_sequence(outputs, batch_first=True)
        # outputs is a bathc*max_enc_steps*(2*hidden_dim), but for the hidden
        # must give then to batch first format, so we need to implement this
        # with the following code.
        hidden_c, hidden_s = hidden
        hidden_c = self.reduce_(hidden_c.permute([1, 0, 2]).contiguous().view(batch_size, -1))
        hidden_s = self.reduce_(hidden_s.permute([1, 0, 2]).contiguous().view(batch_size, -1))
        
        hidden_c.unsqueeze_(1)
        hidden_s.unsqueeze_(1)
        # to make the hidden to be batch first and later, do the loop over time
        hidden = (hidden_c.permute([1,0,2]), hidden_s.permute([1, 0, 2]))
        return outputs, hidden

In [6]:
encoder = Encoder()
# display(features_1.squeeze_(1).shape)
features_1, features_2, features_3, features_4 = encoder_transform(features_1, features_2, features_3, features_4)
outputs, hidden = encoder(features_1, features_2)
display(hidden[0].shape)

torch.Size([1, 10, 50])

In [7]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(config.NUM_WORDS+4, config.embedding_dim)
        self.rnn = nn.LSTM(config.embedding_dim, config.hidden_dim, batch_first=True)
        self.logits = nn.Linear(config.hidden_dim, config.NUM_WORDS+4, bias=False)
        
        
        
    def forward(self, X, hidden):
        """
        using the encoder's hidden state to initiaize the decoder
        input and also use the teaching force in the training mode. here
        TODO: how we tell the real difference between the train, teaching force and
        the evalutaion?
        """
        X = self.embed(X)
        # first we need to transoform the X to be the seq_len first
        X = X.unsqueeze_(1) # just one time step, the X now shoud be batch_size*1*embeeding_dim
        outputs, hidden = self.rnn(X, hidden)
        outputs = F.log_softmax(self.logits(outputs.squeeze_(1)), dim=1)
        return outputs, hidden

In [9]:
decoder = Decoder()
outputs = []
# simply test the time dimension 
# feature 3 shoud be time first
# features_4 = features_4.permute([1, 0])
loss = 0
# display(features_3.squeeze_(1).permute([1, 0]))
critertion = nn.NLLLoss(ignore_index=0)
target = features_4
for di in range(config.max_dec_steps):
    X = features_3[di]
    output, hidden = decoder(X, hidden)
    # here to implement the loss wrt target
    print("+++++++++++++{}++++++++++++++".format(di))
    display(output.shape)
    display(target[di])
    loss += critertion(output, target[di])
    display(loss)


+++++++++++++0++++++++++++++


torch.Size([10, 50004])

tensor([  91, 2441, 2017,   31,   31, 2017,   15, 2250, 1559, 2017])

tensor(10.8721, grad_fn=<AddBackward>)

+++++++++++++1++++++++++++++


...

tensor(421.8301, grad_fn=<ThAddBackward>)

+++++++++++++39++++++++++++++


torch.Size([10, 50004])

tensor([   0,    0, 1336,    0,    0, 1559,    0,    5,  623,    0])

tensor(432.6760, grad_fn=<ThAddBackward>)

In [10]:
from torch.utils.data import DataLoader

train_data = DataLoader(sum_dataset, shuffle=True, batch_size=10)
len(train_data)

14583

In [11]:
for i_batch, sample_batched in enumerate(tqdm(train_data)):
    if i_batch % 100 == 0:
        print("one batch finished.")

  3%|▎         | 455/14583 [00:00<00:06, 2268.89it/s]

one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.


  6%|▋         | 947/14583 [00:00<00:05, 2360.33it/s]

one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.


 10%|▉         | 1454/14583 [00:00<00:05, 2416.65it/s]

...

one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.


 96%|█████████▋| 14039/14583 [00:05<00:00, 2704.27it/s]

one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.
one batch finished.


100%|██████████| 14583/14583 [00:05<00:00, 2720.30it/s]

one batch finished.
one batch finished.
one batch finished.


In [12]:
display(features_1.shape)
display(features_2.shape)
display(features_3.shape)
display(features_4.shape)

torch.Size([10, 600])

torch.Size([10])

torch.Size([40, 10])

torch.Size([40, 10])